## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

Когорта ds_plus 27, Ангелина, ЖЕСТКИЙ ДЕДЛАЙН: 15 февраля.

В колонках датасета содержатся следующие данные:

• longitude — широта;

• latitude — долгота;

• housing_median_age — медианный возраст жителей жилого массива;

• total_rooms — общее количество комнат в домах жилого массива;

• total_bedrooms — общее количество спален в домах жилого массива;

• population — количество человек, которые проживают в жилом массиве;

• households — количество домовладений в жилом массиве;

• median_income — медианный доход жителей жилого массива;

• median_house_value — медианная стоимость дома в жилом массиве;

• ocean_proximity — близость к океану.

# Подготовка данных

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression

1. Инициализируйте локальную Spark-сессию.

In [2]:
spark = SparkSession.builder \
    .appName('HousingPrediction') \
    .getOrCreate()

2. Прочитайте содержимое файла /datasets/housing.csv.

In [3]:
file_path = '/datasets/housing.csv'
df_housing = spark.read.csv(file_path, header=True, inferSchema=True)

3. Выведите типы данных колонок датасета. Используйте методы pySpark.

In [4]:
df_housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



double — тип данных для колонок, содержащих вещественные числа;
string — тип данных для колонок, содержащих текстовые строки.

4. Выполните предобработку данных:

In [5]:
missing_values = {col: df_housing.filter(F.col(col).isNull()).count() for col in df_housing.columns}
print('Имеются следующие пропуски в данных:')
for col, count in missing_values.items():
    print(f'{col}: {count}')

Имеются следующие пропуски в данных:
longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 207
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


In [6]:
mean_total_bedrooms = df_housing.agg(F.mean('total_bedrooms')).collect()[0][0]
df_housing = df_housing.na.fill(mean_total_bedrooms, subset=['total_bedrooms'])

In [7]:
missing_values = {col: df_housing.filter(F.col(col).isNull()).count() for col in df_housing.columns}
print('Имеются следующие пропуски в данных:')
for col, count in missing_values.items():
    print(f'{col}: {count}')

Имеются следующие пропуски в данных:
longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 0
population: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0


Заполняем пропуски средним значением.После преобразований пропусков не осталось.

Преобразуйте колонку с категориальными значениями техникой One hot encoding.

In [8]:
indexer = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_index')
encoder = OneHotEncoder(inputCol='ocean_proximity_index', outputCol='ocean_proximity_encoded')

# Обучение моделей

Подготовка для первой модели:

In [9]:
train_data, test_data = df_housing.randomSplit([0.8, 0.2], seed=42)

In [10]:
numeric_and_encoded_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms',
                            'population', 'households', 'median_income', 'ocean_proximity_encoded']
assembler = VectorAssembler(inputCols=numeric_and_encoded_cols, outputCol='features')

In [11]:
lr_all_data = LinearRegression(featuresCol='features', labelCol='median_house_value')

In [12]:
pipeline_all_data = Pipeline(stages=[indexer, encoder, assembler, lr_all_data])

In [13]:
model_all_data = pipeline_all_data.fit(train_data)
predictions_all_data = model_all_data.transform(test_data)

24/02/11 17:22:34 WARN Instrumentation: [a6b731bd] regParam is zero, which might cause numerical instability and overfitting.
24/02/11 17:22:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/02/11 17:22:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
24/02/11 17:22:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
24/02/11 17:22:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [14]:
evaluator = RegressionEvaluator(labelCol='median_house_value', metricName='rmse')
rmse_all_data = evaluator.evaluate(predictions_all_data)
mae_all_data = evaluator.setMetricName('mae').evaluate(predictions_all_data)
r2_all_data = evaluator.setMetricName('r2').evaluate(predictions_all_data)

In [15]:
print('Результаты модели, использующей все данные:')
print(f'RMSE: {rmse_all_data}')
print(f'MAE: {mae_all_data}')
print(f'R2: {r2_all_data}')

Результаты модели, использующей все данные:
RMSE: 70781.23017920922
MAE: 50855.05025358481
R2: 0.6378987876472269


Подготовка для второй модели

In [16]:
numeric_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms',
                'population', 'households', 'median_income']
assembler_numeric_only = VectorAssembler(inputCols=numeric_cols, outputCol='features_numeric_only')
lr_numeric_only = LinearRegression(featuresCol='features_numeric_only', labelCol='median_house_value')
pipeline_numeric_only = Pipeline(stages=[assembler_numeric_only, lr_numeric_only])

In [17]:
model_numeric_only = pipeline_numeric_only.fit(train_data)
predictions_numeric_only = model_numeric_only.transform(test_data)

24/02/11 17:22:40 WARN Instrumentation: [883f0aed] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
evaluator = RegressionEvaluator(labelCol='median_house_value', metricName='rmse')
rmse_numeric_only = evaluator.evaluate(predictions_numeric_only)
mae_numeric_only = evaluator.setMetricName('mae').evaluate(predictions_numeric_only)
r2_numeric_only = evaluator.setMetricName('r2').evaluate(predictions_numeric_only)

In [19]:
print('Результаты модели, использующей только числовые переменные:')
print(f'RMSE: {rmse_numeric_only}')
print(f'MAE: {mae_numeric_only}')
print(f'R2: {r2_numeric_only}')

Результаты модели, использующей только числовые переменные:
RMSE: 71782.88243199226
MAE: 51787.84444068335
R2: 0.6275778072465257


In [20]:
spark.stop()

# Анализ результатов

Были проведены следующие действия:

1. Инициализируйте локальную Spark-сессию.
2. Прочитайте содержимое файла /datasets/housing.csv.
3. Выведите типы данных колонок датасета. Используйте методы pySpark.
4. Выполните предобработку данных:

• Исследуйте данные на наличие пропусков и заполните их, выбрав значения по своему усмотрению.

• Преобразуйте колонку с категориальными значениями техникой One hot encoding.

5. Постройте две модели линейной регрессии на разных наборах данных:

• Используя все данные из файла;

• Используя только числовые переменные, исключив категориальные.


Для построения модели используйте оценщик LinearRegression из библиотеки MLlib.

6. Сравните результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2. Сделайте выводы.

Результаты модели, использующей все данные:

• RMSE: 70781.23017920922

• MAE: 50855.05025358481

• R2: 0.6378987876472269

Результаты модели, использующей только числовые переменные:

• RMSE: 0.6275778072465257

• MAE: 51787.84444068335

• R2: 0.6275778072465257

Модель, использующая все данные, демонстрирует немного лучшие результаты по метрикам RMSE и MAE, а также немного более высокий коэффициент детерминации (R2) по сравнению с моделью, использующей только числовые переменные.

Можно подвести следующий вывод:

1. При включении всех данных получается лучше учесть имеющуюся информацию в таблицах и улучшить качество предсказаний.
2. По метрикам RMSE и MAE видно, что модель, которая использовала все данные, лучше справилась с точностью предсказаний по сравнению с моделью, использующей только числовые переменные.
3. Обе модели данных можно использовать, но при включении всех доступных переменных в модель может лучше предсказывать.